# Julia Scheme Seperate Tests

In [ ]:
import ..Utils: Symbol2, Sym, symbol_table, _quote,_if,_set,_define,_lambda,_begin,_definemacro,_quasiquote,_unquote,_unquotesplicing
import ..Prep: prep, require, _append, _cons, _let, is_pair, prep_quasiquote, to_string, let2, macro_table, unzip, Callable
import ..Parser: eof_object, tokenizer ,InPort, next_token, quotes, read2, parse2, fix_for_macro, atom
import ..env: Env, ContinuationException, throw_continuation, callcc, LookupError, find, add_globals, global_env
import ..Eval: eval2, Procedure
import ..Repl: load, repl

In [ ]:
using Test

@testset "Scheme Interpreter Tests" begin
    @testset "expand function tests" begin
        @test begin
            result = expand(42)
            println("expand(42) gives the result: $result \n")
            result == 42
        end

        @test begin
            result = expand([_quote, Sym("exp")])
            println("expand([_quote, 'exp']) gives the result: $result \n")
            result == [_quote, Sym("exp")]
        end

        @test begin
            result = expand([_if, Sym("test"), Sym("consequence"), Sym("alternative")])
            println("expand([_if, 'test', 'consequence', 'alternative']) gives the result: $result \n")
            result == [_if, Sym("test"), Sym("consequence"), Sym("alternative")]
        end

        @test begin
            result = expand([_if, Sym("test"), Sym("consequence")])
            println("expand([_if, 'test', 'consequence']) gives the result: $result \n")
            result == [_if, Sym("test"), Sym("consequence"), nothing]
        end

        @test begin
            result = expand([_set, Sym("var"), 42])
            println("expand([_set, Sym('var'), 42]) gives the result: $result \n")
            result == [_set, Sym("var"), 42]
        end

        @test begin
            result = expand([_define, Sym("var"), 42])
            println("expand([_define, Sym('var'), 42]) gives the result: $result \n")
            result == [_define, Sym("var"), 42]
        end

        @test begin
            result = expand([_define, [Sym("f"), Sym("arg")], "body"])
            println("expand([_define, [Sym('f'), Sym('arg')], 'body']) gives the result: $result \n")
            result == [_define, Sym("f"), [_lambda, [Sym("arg")], "body"]]
        end

        @test begin
            result = expand([_begin, 42, Sym("exp")])
            println("expand([_begin, 42, 'exp']) gives the result: $result \n")
            result == [_begin, 42, Sym("exp")]
        end

        @test begin
            result = expand([_lambda, [Sym("arg")], Sym("body")])
            println("expand([_lambda, [Sym('arg')], 'body']) gives the result: $result \n")
            result == [_lambda, [Sym("arg")], Sym("body")]
        end

        @test begin
            result = expand_quasiquote([_unquote, Sym("exp")])
            println("expand_quasiquote([_unquote, 'exp']) gives the result: $result \n")
            result == Sym("exp")
        end


        @test begin
            result = let2([(Sym("x"), 42), (Sym("y"), Sym("exp"))], "body")
            println("let2([[Sym('x'), 42], [Sym('y'), 'exp']], 'body') gives the result: $result \n")
            result == [[_lambda, [Sym("x"), Sym("y")], "body"], 42, Sym("exp")]
        end
    end
end

expand(42) gives the result: 42 

expand([_quote, 'exp']) gives the result: Symbol2[Symbol2("quote"), Symbol2("exp")] 

expand([_if, 'test', 'consequence', 'alternative']) gives the result: Symbol2[Symbol2("if"), Symbol2("test"), Symbol2("consequence"), Symbol2("alternative")] 

expand([_if, 'test', 'consequence']) gives the result: Union{Nothing, Symbol2}[Symbol2("if"), Symbol2("test"), Symbol2("consequence"), nothing] 

expand([_set, Sym('var'), 42]) gives the result: Any[Symbol2("set!"), Symbol2("var"), 42] 

expand([_define, Sym('var'), 42]) gives the result: Any[Symbol2("define"), Symbol2("var"), 42] 

expand([_define, [Sym('f'), Sym('arg')], 'body']) gives the result: Any[Symbol2("define"), Symbol2("f"), Any[Symbol2("lambda"), Symbol2[Symbol2("arg")], "body"]] 

expand([_begin, 42, 'exp']) gives the result: Any[Symbol2("begin"), 42, Symbol2("exp")] 

expand([_lambda, [Sym('arg')], 'body']) gives the result: Any[Symbol2("lambda"), Symbol2[Symbol2("arg")], Symbol2("body")] 

expand

Test.DefaultTestSet("Scheme Interpreter Tests", Any[Test.DefaultTestSet("expand function tests", Any[], 11, false, false)], 0, false, false)

In [ ]:
tests = [
    "(quote (testing 1 (2.0) -3.14e159)) => (testing 1 (2.0) -3.14e+159)",
    "(+ 2 2) => 4",
    "(+ (* 2 100) (* 1 10)) => 210",
    "(if (> 6 5) (+ 1 1) (+ 2 2)) => 2",
    "(if (< 6 5) (+ 1 1) (+ 2 2)) => 4",
    "(define x 3) => nothing",
    "x => 3",
    "(+ x x) => 6",
    "(begin (define x 1) (set! x (+ x 1)) (+ x 1)) => 3",
    "((lambda (x) (+ x x)) 5) => 10",
    "(define twice (lambda (x) (* 2 x))) => nothing",
    "(twice 5) => 10",
    "(define compose (lambda (f g) (lambda (x) (f (g x))))) => nothing",
    "((compose list twice) 5) => (10)",
    "(define repeat (lambda (f) (compose f f))) => nothing",
    "((repeat twice) 5) => 20",
    "((repeat (repeat twice)) 5) => 80",
    "(define fact (lambda (n) (if (<= n 1) 1 (* n (fact (- n 1)))))) => nothing",
    "(fact 3) => 6",
    "(fact 10) => 3628800",
    "(define abs (lambda (n) ((if (> n 0) + -) 0 n))) => nothing",
    "(list (abs -3) (abs 0) (abs 3)) => (3 0 3)",
    "(define combine (lambda (f) (lambda (x y) (if (null? x) (quote ()) (f (list (car x) (car y)) ((combine f) (cdr x) (cdr y))))))) => nothing",
    "(define zip (combine cons)) => nothing",
    "(zip (list 1 2 3 4) (list 5 6 7 8)) => ((1 5) (2 6) (3 7) (4 8))",
    "(define riff-shuffle (lambda (deck) (begin (define take (lambda (n seq) (if (<= n 0) (quote ()) (cons (car seq) (take (- n 1) (cdr seq)))))) (define drop (lambda (n seq) (if (<= n 0) seq (drop (- n 1) (cdr seq))))) (define mid (lambda (seq) (/ (length seq) 2))) ((combine append) (take (mid deck) deck) (drop (mid deck) deck))))) => None",
    "(riff-shuffle (list 1 2 3 4 5 6 7 8)) => (1 5 2 6 3 7 4 8)",
    "((repeat riff-shuffle) (list 1 2 3 4 5 6 7 8)) => (1 3 5 7 2 4 6 8)",
    "(riff-shuffle (riff-shuffle (riff-shuffle (list 1 2 3 4 5 6 7 8)))) => (1 2 3 4 5 6 7 8)",
    "(define (twice x) (* 2 x)) => nothing",
    "(twice 2) => 4",
    "(define lyst (lambda items items)) => nothing",
    "(lyst 1 2 3 (+ 2 2)) => (1 2 3 4)",
    "(if 1 2) => 2",
    "(if (= 3 4) 2) => nothing",
    "(define ((account bal) amt) (set! bal (+ bal amt)) bal) => nothing",
    "(define a1 (account 100)) => nothing",
    "(a1 0) => 100",
    "(a1 10) => 110",
    "(a1 10) => 120",
    "(define (newton guess function derivative epsilon) (define guess2 (- guess (/ (function guess) (derivative guess)))) (if (< (abs (- guess guess2)) epsilon) guess2 (newton guess2 function derivative epsilon))) => None",
    "(define (square-root a) (newton 1 (lambda (x) (- (* x x) a)) (lambda (x) (* 2 x)) 1e-8)) => nothing",
    "(> (square-root 200.) 14.14213) => #t",
    "(< (square-root 200.) 14.14215) => #t",
    "(= (square-root 200.) (sqrt 200.)) => #t",
    "(define (sum-squares-range start end) (define (sumsq-acc start end acc) (if (> start end) acc (sumsq-acc (+ start 1) end (+ (* start start) acc)))) (sumsq-acc start end 0)) => None",
    "(sum-squares-range 1 3000) => 9004500500",
    "(call/cc (lambda (throw) (+ 5 (* 10 (throw 1))))) ;; throw => 1",
    "(call/cc (lambda (throw) (+ 5 (* 10 1)))) ;; do not throw => 15",
    "(call/cc (lambda (throw) (+ 5 (* 10 (call/cc (lambda (escape) (* 100 (escape 3)))))))) ; 1 level => 35",
    "(call/cc (lambda (throw) (+ 5 (* 10 (call/cc (lambda (escape) (* 100 (throw 3)))))))) ; 2 levels => 3",
    "(call/cc (lambda (throw) (+ 5 (* 10 (call/cc (lambda (escape) (* 100 1))))))) ; 0 levels => 1005",
    "(* 1i 1i) => (-1+0i)",
    "(sqrt -1) => 1i",
    "(let ((a 1) (b 2)) (+ a b)) => 3",
#    "(and 1 2 3) => 3",
#    "(and (> 2 1) 2 3) => 3",
#    "(and) => #t",
#    "(and (> 2 1) (> 2 3)) => #f",
    "(define-macro unless (lambda args `(if (not ,(car args)) (begin ,@(cdr args))))) ; test ` => nothing",
    "(unless (= 2 (+ 1 1)) (display 2) 3 4) => nothing",
    "(unless (= 4 (+ 1 1)) 4) => 4",
    "(quote x) => x",
    "(quote (1 2 three)) => (1 2 three)"
]

In [ ]:
println("Parser Tests")
for test in tests
    # Splitting each test case into the input part and the expected output part
    parts = split(test, " => ")
    input_part = String(parts[1])

    println("Parse>>> ", input_part)
    println(parse2(input_part))
    println("")
end

Parser Tests
Parse>>> (quote (testing 1 (2.0) -3.14e159))
Any[Symbol2("quote"), Any[Symbol2("testing"), 1, Any[2.0], -3.14e159]]

Parse>>> (+ 2 2)
Any[Symbol2("+"), 2, 2]

Parse>>> (+ (* 2 100) (* 1 10))
Any[Symbol2("+"), Any[Symbol2("*"), 2, 100], Any[Symbol2("*"), 1, 10]]

Parse>>> (if (> 6 5) (+ 1 1) (+ 2 2))
Any[Symbol2("if"), Any[Symbol2(">"), 6, 5], Any[Symbol2("+"), 1, 1], Any[Symbol2("+"), 2, 2]]

Parse>>> (if (< 6 5) (+ 1 1) (+ 2 2))
Any[Symbol2("if"), Any[Symbol2("<"), 6, 5], Any[Symbol2("+"), 1, 1], Any[Symbol2("+"), 2, 2]]

Parse>>> (define x 3)
Any[Symbol2("define"), Symbol2("x"), 3]

Parse>>> x
Symbol2("x")

Parse>>> (+ x x)
Symbol2[Symbol2("+"), Symbol2("x"), Symbol2("x")]

Parse>>> (begin (define x 1) (set! x (+ x 1)) (+ x 1))
Any[Symbol2("begin"), Any[Symbol2("define"), Symbol2("x"), 1], Any[Symbol2("set!"), Symbol2("x"), Any[Symbol2("+"), Symbol2("x"), 1]], Any[Symbol2("+"), Symbol2("x"), 1]]

Parse>>> ((lambda (x) (+ x x)) 5)
Any[Any[Symbol2("lambda"), Any[Symbol2("x

In [ ]:
println("scm simulation: Tests")
for test in tests
    # Splitting each test case into the input part and the expected output part
    parts = split(test, " => ")
    input_part = String(parts[1])
    output_part = String(parts[2])

    println("jscm>>> ", input_part, "   ; scheme expected output: ", output_part)
    println(eval2(parse2(input_part)))
    println("")
end

scm simulation: Tests
jscm>>> (quote (testing 1 (2.0) -3.14e159))   ; scheme output: (testing 1 (2.0) -3.14e+159)
Any[Symbol2("testing"), 1, Any[2.0], -3.14e159]

jscm>>> (+ 2 2)   ; scheme output: 4
4

jscm>>> (+ (* 2 100) (* 1 10))   ; scheme output: 210
210

jscm>>> (if (> 6 5) (+ 1 1) (+ 2 2))   ; scheme output: 2
2

jscm>>> (if (< 6 5) (+ 1 1) (+ 2 2))   ; scheme output: 4
4

jscm>>> (define x 3)   ; scheme output: nothing
nothing

jscm>>> x   ; scheme output: 3
3

jscm>>> (+ x x)   ; scheme output: 6
6

jscm>>> (begin (define x 1) (set! x (+ x 1)) (+ x 1))   ; scheme output: 3
3

jscm>>> ((lambda (x) (+ x x)) 5)   ; scheme output: 10
10

jscm>>> (define twice (lambda (x) (* 2 x)))   ; scheme output: nothing
nothing

jscm>>> (twice 5)   ; scheme output: 10
10

jscm>>> (define compose (lambda (f g) (lambda (x) (f (g x)))))   ; scheme output: nothing
nothing

jscm>>> ((compose list twice) 5)   ; scheme output: (10)
[10]

jscm>>> (define repeat (lambda (f) (compose f f)))   ; scheme 

In [ ]:
errorTests = [
    "() => SyntaxError (): wrong length",
    "(set! x) => SyntaxError (set! x): wrong length",
    "(define 3 4) => SyntaxError (define 3 4): can define only a symbol",
    "(quote 1 2) => SyntaxError (quote 1 2): wrong length",
    "(if 1 2 3 4) => SyntaxError (if 1 2 3 4): wrong length",
    "(lambda 3 3) => SyntaxError (lambda 3 3): illegal lambda argument list",
    "(lambda (x)) => SyntaxError (lambda (x)): wrong length",
    "(if (= 1 2) (define-macro a 'a) (define-macro a 'b)) => SyntaxError (define-macro a (quote a)): define-macro only allowed at top level",
    "(twice 2 2) => TypeError expected (x), given (2 2)",
    "(let ((a 1) (b 2 3)) (+ a b)) => SyntaxError (let ((a 1) (b 2 3)) (+ a b)): illegal binding list"
]

10-element Vector{String}:
 "() => SyntaxError (): wrong length"
 "(set! x) => SyntaxError (set! x): wrong length"
 "(define 3 4) => SyntaxError (define 3 4): can define only a symbol"
 "(quote 1 2) => SyntaxError (quote 1 2): wrong length"
 "(if 1 2 3 4) => SyntaxError (if 1 2 3 4): wrong length"
 "(lambda 3 3) => SyntaxError (lambda 3 3): illegal lambda argument list"
 "(lambda (x)) => SyntaxError (lambda (x)): wrong length"
 "(if (= 1 2) (define-macro a 'a) (define-m" ⋯ 52 bytes ⋯ "): define-macro only allowed at top level"
 "(twice 2 2) => TypeError expected (x), given (2 2)"
 "(let ((a 1) (b 2 3)) (+ a b)) => SyntaxError (let ((a 1) (b 2 3)) (+ a b)): illegal binding list"

In [ ]:
repl()

Basic Julia Scheme Interpreter
jscm>>> 